In [1]:
import pandas as pd
import numpy as np
import gzip
import string
import json
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
def readJSON(path):
    for l in gzip.open(path, 'r+'):
        d = eval(l)
        u = d['userID']
        try:
            g = d['gameID']
        except Exception as e:
            g = None
        yield u,g,d
    
def parseData(path):
    for l in gzip.open(path, 'r+'):
        yield eval(l)

In [3]:
raw_r = pd.read_csv('../data/original_set/RAW_recipes.csv')
raw_i = pd.read_csv('../data/original_set/RAW_interactions.csv')
finalData = pd.read_csv('../data/original_set/final_data.csv')

In [4]:
pd.read_csv('../data/original_set/PP_recipes.csv')

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"
...,...,...,...,...,...,...,...,...
178260,323143,76862,"[40480, 6444, 1964, 9369, 486, 569, 17551, 40481]","[[8780], [11835, 1762, 4465, 31494], [6812], [...","[40480, 40482, 729, 2525, 715, 485, 26641, 404...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,"[840, 208, 2499, 2683, 1925, 335, 1511]"
178261,149114,145962,"[40480, 17027, 24715, 974, 11877, 40481]","[[6812], [5940], [30645, 4785, 6821], [6953], ...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[2499, 4717, 1168, 6270, 6324, 7040]"
178262,34200,65066,"[40480, 12187, 11434, 1738, 2627, 40481]","[[6167, 20930, 510], [1353], [15022, 6953], [6...","[40480, 40482, 500, 246, 1719, 5024, 240, 2366...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",2,"[2378, 7655, 3219, 2320, 5168, 5319, 4189, 268..."
178263,30618,77358,"[40480, 870, 488, 1325, 519, 2220, 2417, 488, ...","[[12395, 38308, 40118], [3137, 15022], [30878,...","[40480, 40482, 562, 481, 10734, 240, 23667, 58...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[5627, 2807, 5412, 3399, 7979, 1093, 1257, 780..."


In [5]:
raw_r

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
231633,zydeco spice mix,493372,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
231634,zydeco ya ya deviled eggs,308080,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8
231635,cookies by design cookies on a stick,298512,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10


In [6]:
raw_i

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
...,...,...,...,...,...
1132362,116593,72730,2003-12-09,0,Another approach is to start making sauce with...
1132363,583662,386618,2009-09-29,5,These were so delicious! My husband and I tru...
1132364,157126,78003,2008-06-23,5,WOW! Sometimes I don't take the time to rate ...
1132365,53932,78003,2009-01-11,4,Very good! I used regular port as well. The ...


In [7]:
finalData#['ingredient_list']

,recipe_id,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,mean_rating,review_count,ingredient_list,cuisine
0,137739,arriba baked winter squash mexican style,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,5.0,3,"['winter squash', 'mexican seasoning', 'mixed ...",italian
1,31490,a bit different breakfast pizza,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,3.5,4,"['prepared pizza crust', 'sausage patty', 'egg...",italian
2,112140,all in the kitchen chili,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,4.0,1,"['ground beef', 'yellow onions', 'diced tomato...",mexican
3,59389,alouette potatoes,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,4.5,2,"['spreadable cheese with garlic and herbs', 'n...",italian
4,44061,amish tomato ketchup for canning,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,5.0,1,"['tomato juice', 'apple cider vinegar', 'sugar...",southern_us
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231632,486161,zydeco soup,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22,5.0,6,"['celery', 'onion', 'green sweet pepper', 'gar...",cajun_creole
231633,493372,zydeco spice mix,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13,5.0,1,"['paprika', 'salt', 'garlic powder', 'onion po...",cajun_creole
231634,308080,zydeco ya ya deviled eggs,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8,5.0,5,"['hard-cooked eggs', 'mayonnaise', 'dijon must...",cajun_creole
231635,298512,cookies by design cookies on a stick,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10,1.0,1,"['butter', 'eagle brand condensed milk', 'ligh...",southern_us


In [8]:
raw_r = raw_r[raw_r['minutes'] < 300000]
raw_r.drop(columns=['id','contributor_id']).describe()

,minutes,n_steps,n_ingredients
count,231635.000000,231635.000000,231635.000000
mean,123.108144,9.765506,9.051188
std,1977.767905,5.995153,3.734782
min,0.000000,0.000000,1.000000
25%,20.000000,6.000000,6.000000
50%,40.000000,9.000000,9.000000
75%,65.000000,12.000000,11.000000
max,288000.000000,145.000000,43.000000


In [9]:
combined = pd.merge(raw_r,raw_i,how='inner',left_on='id',right_on='recipe_id')
combined

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,user_id,recipe_id,date,rating,review
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,4470,137739,2006-02-18,5,I used an acorn squash and recipe#137681 Swee...
1,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,593927,137739,2010-08-21,5,This was a nice change. I used butternut squas...
2,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,178427,137739,2011-12-05,5,Excellent recipe! I used butternut squash and ...
3,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,28603,31490,2002-08-19,0,"Have not tried this, but it sounds delicious. ..."
4,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,346277,31490,2006-08-27,5,This recipe was wonderful. Instead of using t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132341,zydeco ya ya deviled eggs,308080,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8,844554,308080,2009-10-14,5,These were very good. I meant to add some jala...
1132342,cookies by design cookies on a stick,298512,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10,804234,298512,2008-05-02,1,I would rate this a zero if I could. I followe...
1132343,cookies by design sugar shortbread cookies,298509,20,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[174.9, 14.0, 33.0, 4.0, 4.0, 11.0, 6.0]",5,"['whip sugar and shortening in a large bowl , ...","i've heard of the 'cookies by design' company,...","['granulated sugar', 'shortening', 'eggs', 'fl...",7,866651,298509,2008-06-19,1,This recipe tastes nothing like the Cookies by...
1132344,cookies by design sugar shortbread cookies,298509,20,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[174.9, 14.0, 33.0, 4.0, 4.0, 11.0, 6.0]",5,"['whip sugar and shortening in a large bowl , ...","i've heard of the 'cookies by design' company,...","['granulated sugar', 'shortening', 'eggs', 'fl...",7,1546277,298509,2010-02-08,5,"yummy cookies, i love this recipe me and my sm..."


In [14]:
finalData['calories'] =  [eval(x)[0] for x in finalData['nutrition']]

In [15]:
recipes = finalData[['name','minutes','ingredients','n_ingredients','calories','mean_rating','cuisine']]

In [29]:
samp = recipes.sample(n=10000, random_state=1)
samp.to_csv('sample.csv',index=False)

In [30]:
pd.read_csv('sample.csv')[:10]

,name,minutes,ingredients,n_ingredients,calories,mean_rating,cuisine
0,tomato stuffed roasted red peppers,25,"['red peppers', 'plum tomatoes', 'green onion'...",7,171.0,4.000000,italian
1,almond cake from albufeira portugal,85,"['all-purpose flour', 'granulated sugar', 'bak...",12,440.8,5.000000,southern_us
2,bean burritos simple for camping,8,"['refried beans', 'burrito-size flour tortilla...",6,375.4,5.000000,mexican
3,potato gorgonzola gratin,65,"['baking potatoes', 'salt and black pepper', '...",8,152.0,3.666667,italian
4,orange glazed barbecued chicken,60,"['orange juice', 'salt', 'red pepper flakes', ...",7,660.8,5.000000,southern_us
5,ghost pepper fresh salsa,15,"['tomatoes', 'onion', 'jalapenos', 'cilantro',...",10,64.3,1.000000,mexican
6,musalaydaar baingun,70,"['baby eggplants', 'salt', 'fennel seed', 'cor...",16,482.4,4.000000,indian
7,giada de laurentiis fettuccine alfredo,20,"['heavy cream', 'fresh lemon juice', 'unsalted...",9,757.4,3.500000,italian
8,zesty cajun skirt steak,25,"['barbecue sauce', 'lime zest', 'lime juice', ...",8,303.0,5.000000,cajun_creole
9,penne with rustic lentil sauce,25,"['chicken stock', 'penne pasta', 'virgin olive...",10,710.9,4.714286,italian


In [31]:
test = samp['ingredients'].apply(lambda x: eval(x))

In [32]:
with open('test_set.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(test, filehandle)

In [33]:
with open('test_set.data', 'rb') as filehandle:
    # read the data as binary data stream
    tempSet = pickle.load(filehandle)

In [23]:
tempSet

215141    [red peppers, plum tomatoes, green onion, garl...
3616      [all-purpose flour, granulated sugar, baking p...
19577     [refried beans, burrito-size flour tortillas, ...
164787    [baking potatoes, salt and black pepper, groun...
149652    [orange juice, salt, red pepper flakes, chicke...
                                ...                        
48394     [pork chops, chinese five spice powder, soy sa...
4588      [idaho potatoes, garlic cloves, salt, butter, ...
213283    [italian bread, butter, part-skim mozzarella c...
140086    [white sugar, brown sugar, butter, vegetable o...
211605    [raisins, hot water, all-purpose flour, baking...
Name: ingredients, Length: 30000, dtype: object

In [9]:
features = ['minutes','n_steps','n_ingredients','recipe_id']
x_train,x_test,y_train,y_test = train_test_split(combined[features],combined['rating'],test_size=0.75)

In [10]:
clf = LogisticRegression(class_weight='balanced').fit(x_train,y_train)

In [11]:
pred = clf.predict(x_test)

In [12]:
sum(pred==y_test)/len(pred)

0.053694981513317475

In [14]:
#Creates List For USER ITEM pair and a list for all unique games
UIR = combined[['user_id','recipe_id','rating']].values.tolist()
allRecipes = combined.recipe_id.unique().tolist()
uniqV = {}

#Loops through user item pair and creates dictionaries for each user and all games they have played
for x in UIR:
    if x[0] not in uniqV.keys():
        uniqV[x[0]] = [x[1]]
    else:
        uniqV[x[0]].append(x[1])

#Loops through rest of train data and adds other games users have played
#for x in train:
#    if x['userID'] in uniqV.keys():
#        uniqV[x['userID']].append(x['gameID'])

In [ ]:
unknown = []
    
#Loops through each user in user item list and randomly samples game they have never played and appends to list
for x in range(len(UIR)//2):
    #Finds the games from all games that the user has never played
    no_try = list(set(allRecipes) ^ set(uniqV[UIR[x][0]]))
    
    #Randomly samples a new game
    new_recipe = np.random.choice(no_try,1)[0]
    
    #Checks to see if this game has been sampled before
#    if (UIR[x][0], new_recipe, 0) not in UIR:
        #Appends new sample game as a game the user has not played yet
#        UIV.append((UIV[x][0], new_game, 0))
        
        #Appends sampled game to dictionary so as to say this game has now been selected do not select again
#        uniqV[UIV[x][0]].append(new_game)
    
    #Checks to see if this game has been sampled before
    if (UIR[x][0], new_recipe) not in unknown:
        #Appends new sample game as a game the user has not played yet
        unknown.append((UIR[x][0], new_recipe))
        
        #Appends sampled game to dictionary so as to say this game has now been selected do not select again
        uniqV[UIR[x][0]].append(new_recipe)

In [ ]:
with open('listfile.txt', 'w') as filehandle:
    filehandle.writelines("%s\n" % item for item in unknown)

In [14]:
#user_sample = np.random.choice(combined['user_id'],100)
#item_sample = np.random.choice(combined['recipe_id'],100)
#userItem = list(zip(user_sample,item_sample))

In [51]:
#with open('listfile.data', 'wb') as filehandle:
    # store the data as binary data stream
#    pickle.dump(userItem, filehandle)

In [17]:
# open file and read the content in a list
with open('../testData/listfile.data', 'rb') as filehandle:
    # read the data as binary data stream
    tempList = pickle.load(filehandle)

In [18]:
recipeCount = defaultdict(int)
totalTried = 0

for x in UIR:
    recipeCount[x[1]] += 1
    totalTried += 1

mostPopular = [(recipeCount[x], x) for x in recipeCount]
mostPopular.sort()
mostPopular.reverse()

In [23]:
mostPopular

[(1613, 2886),
 (1601, 27208),
 (1579, 89204),
 (1448, 39087),
 (1322, 67256),
 (1305, 54257),
 (1234, 22782),
 (1220, 32204),
 (997, 69173),
 (904, 68955),
 (877, 33919),
 (855, 82102),
 (847, 25885),
 (802, 28148),
 (786, 135350),
 (770, 26110),
 (762, 99476),
 (731, 10744),
 (730, 129926),
 (727, 33671),
 (720, 63689),
 (714, 150863),
 (701, 97496),
 (685, 107786),
 (672, 5170),
 (652, 32614),
 (647, 95222),
 (644, 38298),
 (627, 77397),
 (610, 131018),
 (596, 29679),
 (592, 87782),
 (591, 4627),
 (577, 73166),
 (570, 50719),
 (565, 66121),
 (562, 141983),
 (561, 37336),
 (560, 28768),
 (540, 43509),
 (528, 76864),
 (528, 60350),
 (517, 150384),
 (516, 27520),
 (516, 2496),
 (510, 89207),
 (507, 31750),
 (507, 9272),
 (489, 92095),
 (486, 78814),
 (482, 132263),
 (473, 76930),
 (472, 53914),
 (465, 15411),
 (461, 8701),
 (455, 73825),
 (441, 90674),
 (431, 15242),
 (428, 80156),
 (428, 32844),
 (427, 261889),
 (424, 200296),
 (411, 63828),
 (410, 31128),
 (404, 71373),
 (400, 70165)

In [22]:
mostPopular.reverse()

In [43]:
return1 = set()
return2 = set()
return3 = set()
return4 = set()
return5 = set()
return6 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalTried*.7: break

count = 0
for ic, i in mostPopular:
    count += ic
    return2.add(i)
    if count > totalTried*.16: break

count = 0
for ic, i in mostPopular:
    count += ic
    return3.add(i)
    if count > totalTried*.03: break

count = 0
for ic, i in mostPopular:
    count += ic
    return4.add(i)
    if count > totalTried*.02: break

count = 0
for ic, i in mostPopular:
    count += ic
    return5.add(i)
    if count > totalTried*.01: break
        
count = 0
for ic, i in mostPopular:
    count += ic
    return6.add(i)
    if count > totalTried*.05: break

In [44]:
#The true values for the validation set
trueTried = [d[2] for d in UIR]

#The predicted value based on a 50th percentile threshold
tryPred = []
#for x in UIR:
#    if x[1] in return5:
#        tryPred.append(1)
#    elif x[1] in return4:
#        tryPred.append(2)
#    elif x[1] in return3:
#        tryPred.append(3)
#    elif x[1] in return2:
#        tryPred.append(4)
#    elif x[1] in return1:
#        tryPred.append(5)
#    else:
#        tryPred.append(0)

for x in userItem:
    if x[1] in return5:
        tryPred.append(1)
    elif x[1] in return4:
        tryPred.append(2)
    elif x[1] in return3:
        tryPred.append(3)
    elif x[1] in return2:
        tryPred.append(4)
    elif x[1] in return1:
        tryPred.append(5)
    else:
        tryPred.append(0)

In [45]:
ifTried = []
for x in userItem:
    if x[1] in uniqV[x[0]]:
        ifTried.append(1)
    else:
        ifTried.append(0)

In [50]:
for x in range(len(userItem)):
    print("Recommended Dish and Rating: " + str(userItem[x][1]) + ", " + str(tryPred[x]))

Recommended Dish and Rating: 58497, 5
Recommended Dish and Rating: 142946, 0
Recommended Dish and Rating: 119559, 0
Recommended Dish and Rating: 395619, 5
Recommended Dish and Rating: 58426, 5
Recommended Dish and Rating: 33919, 2
Recommended Dish and Rating: 359010, 0
Recommended Dish and Rating: 153800, 5
Recommended Dish and Rating: 429609, 5
Recommended Dish and Rating: 82486, 4
Recommended Dish and Rating: 20948, 5
Recommended Dish and Rating: 75644, 5
Recommended Dish and Rating: 146160, 5
Recommended Dish and Rating: 208375, 0
Recommended Dish and Rating: 38924, 0
Recommended Dish and Rating: 286647, 5
Recommended Dish and Rating: 14127, 5
Recommended Dish and Rating: 82216, 0
Recommended Dish and Rating: 325566, 0
Recommended Dish and Rating: 294916, 5
Recommended Dish and Rating: 203279, 5
Recommended Dish and Rating: 247264, 0
Recommended Dish and Rating: 40110, 0
Recommended Dish and Rating: 9110, 5
Recommended Dish and Rating: 55127, 5
Recommended Dish and Rating: 142818, 5

In [30]:
#Finds the True Positive, False Positive, True Negative, and False Negative array values
TP_ = np.logical_and(tryPred, ifTried)
FP_ = np.logical_and(tryPred, np.logical_not(ifTried))
TN_ = np.logical_and(np.logical_not(tryPred), np.logical_not(ifTried))
FN_ = np.logical_and(np.logical_not(tryPred), ifTried)

#Finds the number of True Positive, False Positive, True Negative, and False Negative
TP = sum(TP_)
FP = sum(FP_)
TN = sum(TN_)
FN = sum(FN_)

# accuracy
accuracy = (TP + TN) / (TP + FP + TN + FN)
{'Accuracy':accuracy}

{'Accuracy': 0.32}

In [31]:
correct = 0
for x in range(len(tryPred)):
    if tryPred[x] == ifTried[x]:
        correct += 1
correct/len(tryPred)

0.31

In [20]:
#Finds the True Positive, False Positive, True Negative, and False Negative array values
TP_ = np.logical_and(tryPred, trueTried)
FP_ = np.logical_and(tryPred, np.logical_not(trueTried))
TN_ = np.logical_and(np.logical_not(tryPred), np.logical_not(trueTried))
FN_ = np.logical_and(np.logical_not(tryPred), trueTried)

#Finds the number of True Positive, False Positive, True Negative, and False Negative
TP = sum(TP_)
FP = sum(FP_)
TN = sum(TN_)
FN = sum(FN_)

# accuracy
accuracy = (TP + TN) / (TP + FP + TN + FN)
{'Accuracy':accuracy}

{'Accuracy': 0.6772232162254294}

In [21]:
correct = 0
for x in range(len(tryPred)):
    if tryPred[x] == trueTried[x]:
        correct += 1
correct/len(tryPred)

0.4328526793047355

In [22]:
def Jaccard(user1,user2):
    user1 = set(user1)
    user2 = set(user2)
    numer = len(user1.intersection(user2))
    denom = len(user1.union(user2))
    if denom > 0:
        return numer/denom
    else:
        return 0

def findUsers(id):
    return [d[0] for d in UIR if d[1] == id]

In [23]:
#Creates list unique userID and two dictionaries store validation users and the games they played and all games
#and users that have played them
#ID = set([d[0] for d in UIR])
#TrainRecipe = {}
#userRecipes = {}
#for x in ID:
#    TrainRecipe[x] = []
#for x in UIR:
#    if x['userID'] in TrainRecipe.keys():
#        TrainRecipe[x['userID']].append(x['gameID'])
#for x in allRecipes:
#    userRecipes[x] = findUsers(x)

In [24]:
#with open('userRecipes.json', 'w') as fp:
#    json.dump(userRecipes, fp)
f = open('userRecipes.json')
userRecipes = json.load(f)
f.close()

In [ ]:
jPred = []
for x in userItem:
    similarities = []
    #Finds all users that have played g
    g = userRecipes[str(x[1])]
    for y in uniqV[x[0]]:
        #Finds all users that have played g' and computes Jaccard similarity
        gPrime = userRecipes[str(y)]
        similarities.append(Jaccard(g,gPrime))
    #Make prediction based on whether Jaccard similarity is greater than threshold
    if max(similarities) >= 0.031:
        jPred.append(1)
    else:
        jPred.append(0)
jPred